# Refinitiv API Library Credentials

In [1]:
%config Completer.use_jedi = False

## Set your app key and credentials

In [1]:
APP_KEY                     = 'YOUR_APP_KEY'
RDP_LOGIN                   = 'YOUR_REFINITIV_DATA_PLATFORM_LOGIN'
RDP_PASSWORD                = 'YOUR_REFINITIV_DATA_PLATFORM_PASSWORD'
DEPLOYED_PLATFORM_HOST      = 'THE_HOST:PORT_OF_YOUR_DEPLOYED_PLATFORM'  # ADS hostname:PORT or IP:PORT e.g. 'myADS:15000'
DEPLOYED_PLATFORM_USER_NAME = 'YOUR_USER_NAME_ON_YOUR_DEPLOYED_PLATFORM' # DACS user name 

## Helper functions to define and open and close a session

These function will be used by most of the other tutorials to reduce session related code replication.

Set your default session_type below with **SESSION_TYPE** variable below - valid values are:
- "desktop"
- "platform"
- "deployed"  

This will avoid the need to specif it in all the tutorials.

If you want to override the default you can do so, by calling with the session type e.g. **open_session('desktop')**

When you want to release the session, call **close_session()**.

For further information on session types, refer to **1. Quick Start/1.0 - Sessions.ipynb**

In [ ]:
# Set your Default session type here
SESSION_TYPE = 'platform'

import refinitiv.data as rd
from refinitiv.data._data.legacy import get_default_session, set_default_session
session=None
def open_session(session_type=None):
    if session_type is None:
        session_type = SESSION_TYPE
    session = get_default_session()
    if session is None:
        if session_type == "desktop":
            session = rd.session.desktop.Definition(APP_KEY).get_session()
        elif session_type == "platform":
            session = rd.session.platform.Definition(
                app_key=APP_KEY,
                grant=rd.session.platform.GrantPassword(
                    username=RDP_LOGIN,
                    password=RDP_PASSWORD
                )
            ).get_session()
        elif session_type == "deployed":
            session = rd.session.platform.Definition(
                app_key=APP_KEY,
                deployed_platform_host = DEPLOYED_PLATFORM_HOST,
                deployed_platform_username = DEPLOYED_PLATFORM_USER_NAME
            ).get_session()

    if session is None:
        raise Exception(f"Wrong session_type: {session_type}. It must be ['desktop', 'platform', 'deployed']")
    else:
        set_default_session(session)
        #session.set_log_level(logging.DEBUG)
        session.set_log_level(logging.CRITICAL)
        session.open()
        
def close_session():
    session = get_default_session()
    if session:
        session.close()